In [2]:
import torch
from whisper.audio import (
    FRAMES_PER_SECOND,
    HOP_LENGTH,
    N_FRAMES,
    N_SAMPLES,
    SAMPLE_RATE,
    log_mel_spectrogram,
    pad_or_trim,
    load_audio,
)

In [24]:
audio = load_audio(file='./p232_001.wav')

In [26]:
audio.shape

(27861,)

In [27]:
audio = pad_or_trim(audio)
audio.shape

(480000,)

In [18]:
mel = log_mel_spectrogram(audio)

In [19]:
mel.shape

torch.Size([80, 3000])

In [20]:
mel = mel.unsqueeze(-1)
mel.shape

torch.Size([80, 3000, 1])

In [22]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [29]:
feature = feature_extractor(audio)
feature

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_features': [array([[ 0.2999177 ,  0.40661377,  0.31577927, ..., -0.7433233 ,
        -0.7433233 , -0.7433233 ],
       [ 0.04907542,  0.28438365,  0.12169898, ..., -0.7433233 ,
        -0.7433233 , -0.7433233 ],
       [-0.10164297,  0.16083026,  0.0389452 , ..., -0.7433233 ,
        -0.7433233 , -0.7433233 ],
       ...,
       [-0.7433233 , -0.7433233 , -0.7433233 , ..., -0.7433233 ,
        -0.7433233 , -0.7433233 ],
       [-0.7433233 , -0.7433233 , -0.7433233 , ..., -0.7433233 ,
        -0.7433233 , -0.7433233 ],
       [-0.7433233 , -0.7433233 , -0.7433233 , ..., -0.7433233 ,
        -0.7433233 , -0.7433233 ]], dtype=float32)]}

feature_extractor根本就是在做 log_mel_spectrogram

In [34]:
print(len(feature['input_features']))
print(feature['input_features'][0].shape)

1
(80, 3000)


In [35]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")


config.json: 100%|██████████| 1.94k/1.94k [00:00<00:00, 160kB/s]
model.safetensors: 100%|██████████| 151M/151M [00:14<00:00, 10.5MB/s] 
generation_config.json: 100%|██████████| 1.59k/1.59k [00:00<00:00, 125kB/s]


In [53]:
mel = mel.permute(2,1,0)
mel.shape

torch.Size([1, 80, 3000])

In [51]:
model = model.to('cuda')
mel = mel.to('cuda')

In [55]:
generated_ids = model.generate(mel)

In [57]:
generated_ids.shape

torch.Size([1, 7])

明顯有bug，不知道到底是tokenizer解錯，還是模型本身參數有問題，很有機會是tokenizer的問題  
考慮用whisper自己原本的扣來生，不用huggingface了

In [62]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="en", task="transcribe")
# Decode token ids to text
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

transcription

['� greatgiggling.']